# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

from pyopia.classify import Classify
import pyopia.process
import pyopia.io
import exampledata
from pyopia.pipeline import Pipeline
import pyopia.instrument.holo as holo

In [ ]:
filename, background_file = exampledata.get_example_hologram_and_background()

In [ ]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

datafile_hdf = 'proc/holotest'
model_path = exampledata.get_example_model()
threshold = 0.9

holo_initial_settings = {'pixel_size': 4.4, # pixel size in um
                        'wavelength': 658, # laser wavelength in nm
                        'minZ': 22, # minimum reconstruction distance in mm
                        'maxZ': 60, # maximum reconstruction distance in mm
                        'stepZ': 2} #step size in mm

steps = {'initial': holo.Initial('imbg.pgm', **holo_initial_settings),
         'classifier': Classify(model_path=model_path),
         'load': holo.Load(filename),
         'reconstruct': holo.Reconstruct(stack_clean=0),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps)


In [ ]:
stats = processing_pipeline.run()

In [ ]:

# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

In [ ]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))